# JSON Data Type

The `json` type stores semi-structured data as complete objects. This tutorial covers:

- When to use JSON vs normalized tables
- Defining, inserting, and fetching JSON data

## When to Use JSON

**Good for:**
- Evolving schemas not yet finalized
- Heterogeneous data with varying fields per entry
- Metadata and configuration storage
- Preserving structure from external APIs

**Prefer normalized tables when:**
- Structure is consistent across entries
- You need database-level filtering on specific fields
- Referential integrity matters

JSON fields are stored and retrieved as complete objects. Filter on JSON content in Python after fetching.

In [ ]:
import datajoint as dj

schema = dj.Schema('tutorial_json')

## Table Definition

In [ ]:
@schema
class Equipment(dj.Manual):
    definition = """
    equipment_id : int32
    ---
    name : varchar(100)
    specs=null : json    # flexible specifications
    """

## Inserting JSON Data

Pass Python dicts for JSON fields. Structure can vary between entries:

In [ ]:
Equipment.insert([
    {
        'equipment_id': 1,
        'name': 'Microscope A',
        'specs': {
            'magnification': [10, 40, 100],
            'camera': {'model': 'XR500', 'resolution': [2048, 2048]},
            'calibrated': True,
        },
    },
    {
        'equipment_id': 2,
        'name': 'Electrode Array',
        'specs': {
            'channels': 64,
            'impedance_kohm': 0.5,
            'material': 'tungsten',
        },
    },
    {
        'equipment_id': 3,
        'name': 'Pending Setup',
        'specs': None,  # null allowed
    },
])

## Viewing Data

JSON displays as `json` in previews (like blobs):

In [ ]:
Equipment()

## Fetching JSON Data

JSON deserializes to Python dicts on fetch:

In [ ]:
# Fetch all
for row in Equipment.to_dicts():
    print(f"{row['name']}: {row['specs']}")

In [ ]:
# Fetch one and access nested fields
microscope = (Equipment & {'equipment_id': 1}).fetch1()
specs = microscope['specs']

print(f"Camera model: {specs['camera']['model']}")
print(f"Magnifications: {specs['magnification']}")

## Filtering on JSON Content

Fetch then filter in Python:

In [ ]:
# Find calibrated equipment
calibrated = [
    e for e in Equipment.to_dicts()
    if e['specs'] and e['specs'].get('calibrated')
]
print("Calibrated:", [e['name'] for e in calibrated])

# Find equipment with >32 channels
multi_channel = [
    e for e in Equipment.to_dicts()
    if e['specs'] and e['specs'].get('channels', 0) > 32
]
print("Multi-channel:", [e['name'] for e in multi_channel])

## Updating JSON Data

In [ ]:
# Fetch, modify, delete, reinsert
pending = (Equipment & {'equipment_id': 3}).fetch1()
pending['specs'] = {'type': 'behavioral', 'sensors': ['IR', 'pressure']}

(Equipment & {'equipment_id': 3}).delete(prompt=False)
Equipment.insert1(pending)

(Equipment & {'equipment_id': 3}).fetch1()

## Design Guidelines

| JSON | Normalized Tables |
|------|-------------------|
| Flexible schema | Fixed schema |
| Filter in Python | Filter in SQL |
| No type enforcement | Type-safe |
| No referential integrity | FK constraints |

**Tips:**
- Use JSON for metadata, configs, external API responses
- If you frequently filter on a field, normalize it to a column
- Document expected JSON structure in comments

## Cleanup

In [ ]:
schema.drop(prompt=False)